In [1]:
import spacy
import re
from spacy.tokenizer import Tokenizer
from collections import Counter
from spacy import displacy
from spacy.matcher import Matcher
import textacy

nlp = spacy.load("pl_core_news_sm")

How to Read a Text File

In [2]:
file_name = 'introduction.txt'
introduction_file_text = open(file_name).read()
introduction_file_doc = nlp(introduction_file_text)
# Extract tokens for the given doc
print ([token.text for token in introduction_file_doc])

['Jestem', 'Emanuel', 'Kosowski']


Sentence Detection

In [12]:
about_text=('„Divide et impera!”: dziel, a będziesz rządził; dziel a staniesz się bogaty; dziel a oszukasz ludzi, zaślepisz ich rozum, drwić będziesz ze sprawiedliwości. Sprawiedliwość i władza – słowa nie do pogodzenia.')
about_doc = nlp(about_text)
sentences = list(about_doc.sents)
print(len(sentences))
for sentence in sentences:
    print(sentence)

3
„Divide et impera!”
: dziel, a będziesz rządził; dziel a staniesz się bogaty; dziel a oszukasz ludzi, zaślepisz ich rozum, drwić będziesz ze sprawiedliwości.
Sprawiedliwość i władza – słowa nie do pogodzenia.


Here’s an example, where an ellipsis(...) is used as the delimiter:

In [5]:
def set_custom_boundaries(doc):
    for token in doc[:-1]:
        if token.text == '...' or token.text == 'np.' or token.text == 'm.in.':
            doc[token.i+1].is_sent_start = True
    return doc

ellipsis_text = 'ale też, bez przesady… można z tym żyć, tylko…ja bym tak na przykład…'

custom_nlp = spacy.load("pl_core_news_sm")
custom_nlp.add_pipe(set_custom_boundaries, before='parser')
custom_ellipsis_doc = custom_nlp(ellipsis_text)
custom_ellipsis_sentences = list(custom_ellipsis_doc.sents)
for sentence in custom_ellipsis_sentences:
    print(sentence)

ale też, bez przesady…
można z tym żyć, tylko…
ja bym tak na przykład…


In [6]:
ellipsis_doc = nlp(ellipsis_text)
ellipsis_sentences = list(ellipsis_doc.sents)
for sentence in ellipsis_sentences:
   print(sentence)

ale też, bez przesady…
można z tym żyć, tylko…
ja bym tak na przykład…


Tokenization in spaCy

In [7]:
for token in about_doc:
    print (token, token.idx)

„ 0
Divide 1
et 8
impera 11
! 17
” 18
: 19
dziel 21
, 26
a 28
będziesz 30
rządził 39
; 46
dziel 48
a 54
staniesz 56
się 65
bogaty 69
; 75
dziel 77
a 83
oszukasz 85
ludzi 94
, 99
zaślepisz 101
ich 111
rozum 115
, 120
drwić 122
będziesz 128
ze 137
sprawiedliwości 140
. 155
Sprawiedliwość 157
i 172
władza 174
– 181
słowa 183
nie 189
do 193
pogodzenia 196
. 206


In [8]:
for token in about_doc:
    print (token, token.idx, token.text_with_ws,
    token.is_alpha, token.is_punct, token.is_space,
    token.shape_, token.is_stop)

„ 0 „ False True False „ False
Divide 1 Divide  True False False Xxxxx False
et 8 et  True False False xx False
impera 11 impera True False False xxxx False
! 17 ! False True False ! False
” 18 ” False True False ” False
: 19 :  False True False : False
dziel 21 dziel True False False xxxx False
, 26 ,  False True False , False
a 28 a  True False False x True
będziesz 30 będziesz  True False False xxxx False
rządził 39 rządził True False False xxxx False
; 46 ;  False True False ; False
dziel 48 dziel  True False False xxxx False
a 54 a  True False False x True
staniesz 56 staniesz  True False False xxxx False
się 65 się  True False False xxx True
bogaty 69 bogaty True False False xxxx False
; 75 ;  False True False ; False
dziel 77 dziel  True False False xxxx False
a 83 a  True False False x True
oszukasz 85 oszukasz  True False False xxxx False
ludzi 94 ludzi True False False xxxx False
, 99 ,  False True False , False
zaślepisz 101 zaślepisz  True False False xxxx False
ich 111 ich

spaCy allows you to customize tokenization by updating the tokenizer property on the nlp object:

In [14]:
custom_nlp = spacy.load('pl_core_news_sm')
prefix_re = spacy.util.compile_prefix_regex(custom_nlp.Defaults.prefixes)
suffix_re = spacy.util.compile_suffix_regex(custom_nlp.Defaults.suffixes)
infix_re = re.compile(r'''[-~]''')

def customize_tokenizer(nlp):
# Adds support to use `-` as the delimiter for tokenization
    return Tokenizer(nlp.vocab, prefix_search=prefix_re.search,
                    suffix_search=suffix_re.search,
                    infix_finditer=infix_re.finditer,
                    token_match=None
                   )


custom_nlp.tokenizer = customize_tokenizer(custom_nlp)
custom_tokenizer_about_doc = custom_nlp(about_text)
print([token.text for token in custom_tokenizer_about_doc])

['„', 'Divide', 'et', 'impera', '!', '”', ':', 'dziel', ',', 'a', 'będziesz', 'rządził', ';', 'dziel', 'a', 'staniesz', 'się', 'bogaty', ';', 'dziel', 'a', 'oszukasz', 'ludzi', ',', 'zaślepisz', 'ich', 'rozum', ',', 'drwić', 'będziesz', 'ze', 'sprawiedliwości', '.', 'Sprawiedliwość', 'i', 'władza', '–', 'słowa', 'nie', 'do', 'pogodzenia', '.']


Stop words are the most common words in a language. 

In [15]:
import spacy
spacy_stopwords = spacy.lang.pl.stop_words.STOP_WORDS
len(spacy_stopwords)
for stop_word in list(spacy_stopwords)[:10]:
    print(stop_word)

sam
nie
będzie
jakichś
ok
mimo
tej
oni
bylo
moja


You can remove stop words from the input text:

In [16]:
for token in about_doc:
    if not token.is_stop:
        print(token)

„
Divide
et
impera
!
”
:
dziel
,
będziesz
rządził
;
dziel
staniesz
bogaty
;
dziel
oszukasz
ludzi
,
zaślepisz
rozum
,
drwić
będziesz
sprawiedliwości
.
Sprawiedliwość
władza
–
słowa
pogodzenia
.


You can also create a list of tokens not containing stop words:

In [18]:
about_no_stopword_doc = [token for token in about_doc if not token.is_stop]
print (about_no_stopword_doc)

[„, Divide, et, impera, !, ”, :, dziel, ,, będziesz, rządził, ;, dziel, staniesz, bogaty, ;, dziel, oszukasz, ludzi, ,, zaślepisz, rozum, ,, drwić, będziesz, sprawiedliwości, ., Sprawiedliwość, władza, –, słowa, pogodzenia, .]


Lemmatization

In [4]:
conference_help_text = ('Mięso pokroić w kostkę. Cebulę pokroić w kosteczkę i zeszklić na oleju w dużym garnku. Dodać mięso i dokładnie je obsmażyć. '
'Wlać 2 szklanki gorącego bulionu lub wody z solą i pieprzem, zagotować. Następnie dodać połamane suszone grzyby, przykryć, zmniejszyć ogień i gotować przez ok. 45 minut.'
'Dodać listek laurowy, ziela angielskie, kminek, majeranek, powidła śliwkowe lub posiekane śliwki, obrane i pokrojone w kosteczkę obrane jabłko i wymieszać.'
'Dodać odciśniętą kiszoną kapustę oraz wlać szklankę wody, wymieszać. Przykryć i gotować przez ok. 15 minut.'
'Kiełbasę obrać ze skóry, pokroić w kostkę i podsmażyć na patelni. Dodać do kapusty i gotować przez ok. 30 minut. Pod koniec dodać koncentrat pomidorowy.')
conference_help_doc = nlp(conference_help_text)
for token in conference_help_doc:
    print (token, token.lemma_)

Mięso mięso
pokroić pokroić
w w
kostkę kostka
. .
Cebulę cebula
pokroić pokroić
w w
kosteczkę kosteczka
i i
zeszklić zeszklić
na na
oleju olej
w w
dużym duży
garnku garnek
. .
Dodać dodać
mięso mięso
i i
dokładnie dokładnie
je on
obsmażyć obsmażyć
. .
Wlać wlać
2 2
szklanki szklanka
gorącego gorący
bulionu bulion
lub lub
wody woda
z z
solą sól
i i
pieprzem pieprz
, ,
zagotować zagotować
. .
Następnie następnie
dodać dodać
połamane połamać
suszone suszyć
grzyby grzyb
, ,
przykryć przykryć
, ,
zmniejszyć zmniejszyć
ogień ogień
i i
gotować gotować
przez przez
ok ok
. .
45 45
minut minuta
. .
Dodać dodać
listek listek
laurowy laurowy
, ,
ziela ziele
angielskie angielski
, ,
kminek kminek
, ,
majeranek majeranek
, ,
powidła powidło
śliwkowe śliwkowy
lub lub
posiekane posiekać
śliwki śliwka
, ,
obrane obrać
i i
pokrojone pokroić
w w
kosteczkę kosteczka
obrane obrać
jabłko jabłko
i i
wymieszać wymieszać
. .
Dodać dodać
odciśniętą odcisnąć
kiszoną kisić
kapustę kapusta
oraz oraz
wlać wlać
szkl

Word Frequency - 5 commonly occurring words with their frequencies

In [9]:
complete_text = 'Mięso pokroić w kostkę. Cebulę pokroić w kosteczkę i zeszklić na oleju w dużym garnku. Dodać mięso i dokładnie je obsmażyć. ' \
                'Wlać 2 szklanki gorącego bulionu lub wody z solą i pieprzem, zagotować. Następnie dodać połamane suszone grzyby, przykryć, zmniejszyć ogień i gotować przez ok. 45 minut.'\
                'Dodać listek laurowy, ziela angielskie, kminek, majeranek, powidła śliwkowe lub posiekane śliwki, obrane i pokrojone w kosteczkę obrane jabłko i wymieszać.'\
                'Dodać odciśniętą kiszoną kapustę oraz wlać szklankę wody, wymieszać. Przykryć i gotować przez ok. 15 minut.'\
                'Kiełbasę obrać ze skóry, pokroić w kostkę i podsmażyć na patelni. Dodać do kapusty i gotować przez ok. 30 minut. Pod koniec dodać koncentrat pomidorowy.'
complete_doc = nlp(complete_text)

words = [token.text for token in complete_doc
          if not token.is_stop and not token.is_punct]
word_freq = Counter(words)
common_words = word_freq.most_common(5)
print (common_words)

[('Dodać', 4), ('pokroić', 3), ('gotować', 3), ('minut', 3), ('kostkę', 2)]


Comparison with Word Frequency and StopWords the same text with stop words:

In [10]:
words_all = [token.text for token in complete_doc if not token.is_punct]
word_freq_all = Counter(words_all)
common_words_all = word_freq_all.most_common(5)
print (common_words_all)

[('i', 9), ('w', 5), ('Dodać', 4), ('pokroić', 3), ('gotować', 3)]


Part of Speech Tagging - In spaCy, POS tags are available as an attribute on the Token object:

In [13]:
for token in about_doc:
    print (token, token.tag_, token.pos_, spacy.explain(token.tag_))

„ INTERP PUNCT None
Divide XXX X None
et XXX X None
impera XXX X None
! INTERP PUNCT None
” INTERP PUNCT None
: INTERP PUNCT None
dziel IMPT VERB None
, INTERP PUNCT None
a CONJ CCONJ conjunction
będziesz BEDZIE VERB None
rządził PRAET VERB None
; INTERP PUNCT None
dziel SUBST NOUN None
a CONJ CCONJ conjunction
staniesz FIN VERB None
się QUB PART None
bogaty ADJ ADJ adjective
; INTERP PUNCT None
dziel SUBST NOUN None
a CONJ CCONJ conjunction
oszukasz FIN VERB None
ludzi SUBST NOUN None
, INTERP PUNCT None
zaślepisz FIN VERB None
ich PPRON3 PRON None
rozum SUBST NOUN None
, INTERP PUNCT None
drwić INF VERB None
będziesz BEDZIE VERB None
ze PREP ADP None
sprawiedliwości SUBST NOUN None
. INTERP PUNCT None
Sprawiedliwość SUBST NOUN None
i CONJ CCONJ conjunction
władza SUBST NOUN None
– INTERP PUNCT None
słowa SUBST NOUN None
nie QUB PART None
do PREP ADP None
pogodzenia GER NOUN None
. INTERP PUNCT None


Using POS tags, you can extract a particular category of words:

In [18]:
nouns = []
adjectives = []
for token in about_doc:
     if token.pos_ == 'NOUN':
         nouns.append(token)
     if token.pos_ == 'ADJ':
         adjectives.append(token)

nouns

[dziel,
 dziel,
 ludzi,
 rozum,
 sprawiedliwości,
 Sprawiedliwość,
 władza,
 słowa,
 pogodzenia]

In [19]:
adjectives

[bogaty]

Visualization: Using displaCy

In [26]:
about_interest_text = ('bulion – mocny rosół, odtłuszczony i podawany na ciepło lub zimno;'
     'bulion – wywar z kości, mięsa, warzyw, służący do gotowania lub duszenia potraw, zup, sosów;.'
     'bulion – koncentrat mięsny lub warzywny, do rozpuszczenia w wodzie.')
about_interest_doc = nlp(about_interest_text)
displacy.render(about_interest_doc, style='dep', jupyter=True)

Preprocessing Functions

In [27]:
def is_token_allowed(token):
     if (not token or not token.string.strip() or
         token.is_stop or token.is_punct):
         return False
     return True

def preprocess_token(token):

     return token.lemma_.strip().lower()

complete_filtered_tokens = [preprocess_token(token)
    for token in complete_doc if is_token_allowed(token)]
complete_filtered_tokens

['mięso',
 'pokroić',
 'kostka',
 'cebula',
 'pokroić',
 'kosteczka',
 'zeszklić',
 'olej',
 'duży',
 'garnek',
 'dodać',
 'mięso',
 'dokładnie',
 'obsmażyć',
 'wlać',
 '2',
 'szklanka',
 'gorący',
 'bulion',
 'woda',
 'sól',
 'pieprz',
 'zagotować',
 'następnie',
 'dodać',
 'połamać',
 'suszyć',
 'grzyb',
 'przykryć',
 'zmniejszyć',
 'ogień',
 'gotować',
 '45',
 'minuta',
 'dodać',
 'listek',
 'laurowy',
 'ziele',
 'angielski',
 'kminek',
 'majeranek',
 'powidło',
 'śliwkowy',
 'posiekać',
 'śliwka',
 'obrać',
 'pokroić',
 'kosteczka',
 'obrać',
 'jabłko',
 'wymieszać',
 'dodać',
 'odcisnąć',
 'kisić',
 'kapusta',
 'wlać',
 'szklanka',
 'woda',
 'wymieszać',
 'przykryć',
 'gotować',
 '15',
 'minuta',
 'kiełbasa',
 'obrać',
 'skóra',
 'pokroić',
 'kostka',
 'podsmażyć',
 'patelnia',
 'dodać',
 'kapusta',
 'gotować',
 '30',
 'minuta',
 'koniec',
 'dodać',
 'koncentrat',
 'pomidorowy']

Rule-Based Matching Using spaCy

With rule-based matching, you can extract a first name and a last name, which are always proper nouns:

In [40]:
matcher = Matcher(nlp.vocab)
text_surname = 'Mam na imię Adam Smith i mieszkam w Gdyni'
nlp_nazwisko = nlp(text_surname)
def extract_full_name(nlp_doc):
     pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
     matcher.add('FULL_NAME', None, pattern)
     matches = matcher(nlp_doc)
     for match_id, start, end in matches:
         span = nlp_nazwisko[start:end]
         return span.text

extract_full_name(nlp_nazwisko)

You can also use rule-based matching to extract phone numbers:

In [38]:
matcher = Matcher(nlp.vocab)
conference_org_text = ('Mój aktualny'
    'nr telefonu to'
    'dom: 694202137'
    'stacjonarny: (123) 456-789')
def extract_phone_number(nlp_doc):
    pattern = [{'ORTH': '('}, {'SHAPE': 'ddd'},
               {'ORTH': ')'}, {'SHAPE': 'ddd'},
               {'ORTH': '-', 'OP': '?'},
               {'SHAPE': 'ddd'}]
    matcher.add('PHONE_NUMBER', None, pattern)
    matches = matcher(nlp_doc)
    for match_id, start, end in matches:
        span = nlp_doc[start:end]
        return span.text
conference_org_doc = nlp(conference_org_text)
extract_phone_number(conference_org_doc)

'(123) 456-789'

Dependency Parsing Using spaCy

In [41]:
text = 'Emanuel jest dzikiem SW'
doc = nlp(text)
for token in doc:
   print (token.text, token.tag_, token.head.text, token.dep_)

Emanuel SUBST dzikiem nsubj
jest FIN dzikiem cop
dzikiem SUBST dzikiem ROOT
SW SUBST dzikiem nmod


There is a detailed list of relationships with descriptions. You can use displaCy to visualize the dependency tree:

In [43]:
displacy.serve(doc, style='dep')


Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


Navigating the Tree and Subtree

spaCy provides attributes like children, lefts, rights, and subtree to navigate the parse tree:

In [46]:
one_line_about_text = ('Emanuel Kosowski jest dzikiem SW'
   'Holduje kozak backlevery w fullu')
one_line_about_doc = nlp(one_line_about_text)
print([token.text for token in one_line_about_doc[5].children])
print (one_line_about_doc[5].nbor(-1))
print (one_line_about_doc[5].nbor())
print([token.text for token in one_line_about_doc[5].lefts])
print([token.text for token in one_line_about_doc[5].rights])
print (list(one_line_about_doc[5].subtree))

['backlevery']
SWHolduje
backlevery
[]
['backlevery']
[kozak, backlevery, w, fullu]


You can construct a function that takes a subtree as an argument and returns a string by merging words in it:

In [47]:
def flatten_tree(tree):
   return ''.join([token.text_with_ws for token in list(tree)]).strip()
print (flatten_tree(one_line_about_doc[5].subtree))

kozak backlevery w fullu


Shallow Parsing

spaCy has the property noun_chunks on Doc object. You can use it to extract noun phrases:

In [50]:
conference_text = ('Nadchodzą targi Gamedev wraz GameJam'
    'Odbywać się będą na HSW w Gdyni, 19 maja 2021')
conference_doc = nlp(conference_text)
for chunk in conference_doc.noun_chunks:
    print (chunk)

Verb Phrase Detection

In [3]:
about_talk_text = ('Ten tekst pokaże zastosowanie'
                   'zastosowanie Natural Language Processing w'
                   'w textacy')
pattern = r'(<VERB>?<ADV>*<VERB>+)'
about_talk_doc = textacy.make_spacy_doc(about_talk_text,
                                        lang='pl_core_news_sm')
verb_phrases = textacy.extract.pos_regex_matches(about_talk_doc, pattern)
for chunk in verb_phrases:
    print(chunk.text)

pokaże


c:\users\emanuel\appdata\local\programs\python\python39\lib\site-packages\textacy\extract.py:334: DeprecationWarning: `pos_regex_matches()` has been deprecated! for similar but more powerful and performant functionality, use `textacy.extract.matches()` instead.
  utils.deprecated(


In [5]:
for chunk in about_talk_doc.noun_chunks:
    print (chunk)

Named Entity Recognition

spaCy has the property ents on Doc objects. You can use it to extract named entities:

In [7]:
class_text = ('Mięso pokroić w kostkę. Cebulę pokroić w kosteczkę i zeszklić na oleju w dużym garnku. Dodać mięso i dokładnie je obsmażyć. ' 
                'Wlać 2 szklanki gorącego bulionu lub wody z solą i pieprzem, zagotować. Następnie dodać połamane suszone grzyby, przykryć, zmniejszyć ogień i gotować przez ok. 45 minut.'
                'Dodać listek laurowy, ziela angielskie, kminek, majeranek, powidła śliwkowe lub posiekane śliwki, obrane i pokrojone w kosteczkę obrane jabłko i wymieszać.')
class_doc = nlp(class_text)
for ent in class_doc.ents:
    print(ent.text, ent.start_char, ent.end_char,
         ent.label_, spacy.explain(ent.label_))

In [ ]:
displacy.serve(class_doc, style='ent')

c:\users\emanuel\appdata\local\programs\python\python39\lib\site-packages\spacy\displacy\__init__.py:94: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)
c:\users\emanuel\appdata\local\programs\python\python39\lib\site-packages\spacy\displacy\__init__.py:189: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

